In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

from google.colab import files

Mounted at /content/gdrive


In [ ]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"paulinakaczyska","key":"a590f81fa200cd408d5cb174df96882a"}'}

In [ ]:
#make sure kaggle.json file is present
!ls -lha kaggle.json

#Install kaggle API client
!pip install -q kaggle

#kaggle API client expects the file to be in ~/.kaggle

#so move it there
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

#seting permissions
!chmod 600 /root/.kaggle/kaggle.json

#check your directory before downloading the datasets
!pwd

#downloading
!kaggle datasets download -d wenruliu/adult-income-dataset

#unzipping
!unzip adult-income-dataset.zip

-rw-r--r-- 1 root root 71 Dec  8 11:58 kaggle.json
/content
  0% 0.00/652k [00:00<?, ?B/s]
100% 652k/652k [00:00<00:00, 113MB/s]
Archive:  adult-income-dataset.zip
  inflating: adult.csv               


In [ ]:

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer

import matplotlib.pyplot as plt
import xgboost

In [ ]:
!pip install dalex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 5.2 MB/s 
  Created wheel for dalex: filename=dalex-1.5.0-py3-none-any.whl size=1043321 sha256=d93dbe2a07c024d1a112e1e058d2ff475bf172bc9c7bf6c2c20ad8b585d3e901
  Stored in directory: /root/.cache/pip/wheels/b1/02/58/77ac4cb307fec9f3324c6aa4f9f23a7b0e886e313ebc280257
Successfully built dalex


In [ ]:
import dalex as dx

In [ ]:
adult = pd.read_csv("adult.csv", encoding = "UTF-8")

In [ ]:
categorical = ['workclass','education','marital-status','occupation','relationship','race','gender','native-country','income']
cont = ['age','fnlwgt','educational-num', 'capital-gain', 'capital-loss', 'hours-per-week']
adult_i_c = pd.get_dummies(adult[categorical])
y = adult_i_c['income_>50K']
x = pd.concat([adult_i_c.drop(labels = ['income_>50K','income_<=50K'],axis=1),adult[cont]],axis =1)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.2)

In [ ]:
X_train

,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,education_10th,...,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week
38768,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,39,123983,13,0,0,40
26887,0,0,0,0,1,0,0,0,0,0,...,0,1,0,0,50,158284,10,0,0,40
36571,0,0,0,0,1,0,0,0,0,0,...,0,1,0,0,46,403911,9,0,1902,40
27632,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,49,201498,9,0,0,40
10515,0,0,0,0,1,0,0,0,0,0,...,0,1,0,0,31,198452,10,0,0,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34588,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,33,419895,3,0,0,40
44150,0,0,0,0,1,0,0,0,0,0,...,0,1,0,0,26,192208,9,0,0,32
2885,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,40,320451,13,0,0,40
45138,0,0,0,0,1,0,0,0,0,0,...,0,1,0,0,48,149210,9,0,0,45


#Training first model

In [ ]:
model_xgb = xgboost.XGBClassifier(
    n_estimators=50, 
    max_depth=2, 
    use_label_encoder=False, 
    eval_metric="logloss",
    
    enable_categorical=True,
    tree_method="hist"
)

model_xgb.fit(X_train, y_train)


XGBClassifier(enable_categorical=True, eval_metric='logloss', max_depth=2,
              n_estimators=50, tree_method='hist', use_label_encoder=False)

In [ ]:

def pf_xgboost_classifier_categorical(model, df):
    df.loc[:, df.dtypes == 'object'] =\
        df.select_dtypes(['object'])\
        .apply(lambda x: x.astype('category'))
    return model.predict_proba(df)[:, 1]

explainer = dx.Explainer(model_xgb, X_test, y_test, predict_function=pf_xgboost_classifier_categorical)

Preparation of a new explainer is initiated

  -> data              : 9769 rows 108 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 9769 values
  -> model_class       : xgboost.sklearn.XGBClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function pf_xgboost_classifier_categorical at 0x7f3a19d86940> will be used
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.0166, mean = 0.241, max = 0.969
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.889, mean = -0.00231, max = 0.983
  -> model_info        : package xgboost

A new explainer has been created!


In [ ]:
explainer.model_performance()

,recall,precision,f1,accuracy,auc
XGBClassifier,0.539417,0.805502,0.646138,0.858839,0.912263


In [ ]:

print('Fairness based on gender:')
protected_variable = X_test.gender_Male.apply(lambda x: "male" if x else "female")
privileged_group = "male"

fobject = explainer.model_fairness(
    protected=protected_variable,
    privileged=privileged_group
)
fobject.fairness_check()
fobject.plot()
print('Fairness based on race:')
protected2 = adult['race'].iloc[X_test.index]
privileged2 = 'White'

fobject2 = explainer.model_fairness(
    protected=protected2,
    privileged=privileged2
)
fobject2.fairness_check()
fobject2.plot()
print('Fairness based on age:')
protected3 = np.where(adult.iloc[X_test.index]['age']<30,'young','old')
privileged3 = 'old'

fobject3 = explainer.model_fairness(
    protected=protected3,
    privileged=privileged3
)
fobject3.fairness_check()
fobject3.plot()

Fairness based on gender:
Bias detected in 2 metrics: FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'male'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
             TPR       ACC       PPV       FPR       STP
female  0.858696  1.127427  1.023661  0.220339  0.318841


Fairness based on race:
Bias detected in 4 metrics: TPR, PPV, FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'White'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
                         TPR       ACC       PPV       FPR       STP
Amer-Indian-Eskimo  0.795918  1.065805  1.056720  0.279070  0.426036
Asian-Pac-Islander  1.280148  0.975323  0.850801  2.720930  1.621302
Black               0.838590  1.085781  1.050555  0.255814  0.378698
Other               0.695733  1.074031  0.739827  0.627907  0.366864


Fairness based on age:
Bias detected in 3 metrics: TPR, FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'old'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
            TPR      ACC       PPV       FPR     STP
young  0.434938  1.17402  1.129514  0.015385  0.0625


#Training and describing second model

In [ ]:
from sklearn.neural_network import MLPClassifier
model_mlp = MLPClassifier((100,100))


def mlp_classifier_categorical(model, df):
    df.loc[:, df.dtypes == 'object'] =\
        df.select_dtypes(['object'])\
        .apply(lambda x: x.astype('category'))
    return model.predict_proba(df)[:, 1]


model_mlp.fit(X_train, y_train)
explainer2 = dx.Explainer(model_mlp, X_test, y_test, predict_function=pf_xgboost_classifier_categorical)

Preparation of a new explainer is initiated

  -> data              : 9769 rows 108 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 9769 values
  -> model_class       : sklearn.neural_network._multilayer_perceptron.MLPClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function pf_xgboost_classifier_categorical at 0x7f3a1fc57670> will be used
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.0013, mean = 0.209, max = 1.0
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -1.0, mean = 0.0302, max = 0.987
  -> model_info        : package sklearn

A new explainer has been created!


In [ ]:
explainer2.model_performance()

,recall,precision,f1,accuracy,auc
MLPClassifier,0.454584,0.785926,0.576004,0.840106,0.897048


In [ ]:

print('Fairness based on gender:')
protected_variable = X_test.gender_Male.apply(lambda x: "male" if x else "female")
privileged_group = "male"

fobject_MLP = explainer2.model_fairness(
    protected=protected_variable,
    privileged=privileged_group
)
fobject_MLP.fairness_check()
fobject_MLP.plot(objects=[fobject])
print('Fairness based on race:')
protected2 = adult['race'].iloc[X_test.index]
privileged2 = 'White'

fobject2_MLP = explainer2.model_fairness(
    protected=protected2,
    privileged=privileged2
)
fobject2_MLP.fairness_check()
fobject2_MLP.plot(objects=[fobject2])
print('Fairness based on age:')
protected3 = np.where(adult.iloc[X_test.index]['age']<30,'young','old')
privileged3 = 'old'

fobject3_MLP = explainer2.model_fairness(
    protected=protected3,
    privileged=privileged3
)
fobject3_MLP.fairness_check()
fobject3_MLP.plot(objects=[fobject3])

Fairness based on gender:
Bias detected in 3 metrics: TPR, FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'male'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
             TPR       ACC       PPV       FPR       STP
female  0.718487  1.134328  0.938131  0.277778  0.287293


Fairness based on race:
Bias detected in 4 metrics: TPR, PPV, FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'White'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
                         TPR       ACC       PPV       FPR       STP
Amer-Indian-Eskimo  0.624454  1.015625  0.561315  1.463415  0.632653
Asian-Pac-Islander  1.355895  0.997596  0.910240  2.219512  1.612245
Black               0.646288  1.090144  1.024020  0.243902  0.299320
Other               0.545852  1.112981  1.264223       NaN  0.170068

Warning!
Take into consideration that NaN's are present, consider checking 'metric_scores' plot to see the difference


Fairness based on age:
Bias detected in 4 metrics: TPR, PPV, FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'old'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
            TPR       ACC       PPV       FPR       STP
young  0.587607  1.199495  0.714643  0.218182  0.139037


#Improving fairness

In [ ]:
from dalex.fairness import resample, reweight, roc_pivot
from copy import copy


# resample

protected_variable_t = X_train.gender_Male.apply(lambda x: "male" if x else "female")
indices_resample = resample(
    protected_variable_t, 
    y_train, 
    type='preferential', # uniform
    probs=model_xgb.predict_proba(X_train)[:, 1], # requires probabilities
    verbose=False
)
model_resample = copy(model_xgb)
model_resample.fit(X_train.iloc[indices_resample, :], y_train.iloc[indices_resample])
explainer_resample = dx.Explainer(
    model_resample, 
    X_test, 
    y_test, 
    label='XGBClassifier with Resample mitigation',
    verbose=False
)

protected_variable = X_test.gender_Male.apply(lambda x: "male" if x else "female")
privileged_group = "male"

fobject_resample = explainer_resample.model_fairness(
    protected=protected_variable,
    privileged=privileged_group
)
fobject_resample.fairness_check()


Bias detected in 4 metrics: TPR, PPV, FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'male'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
             TPR       ACC       PPV       FPR       STP
female  2.064067  1.120755  0.570321  5.235294  1.358333


In [ ]:
explainer_resample.model_performance()

,recall,precision,f1,accuracy,auc
XGBClassifier with Resample mitigation,0.419452,0.745053,0.536732,0.827004,0.890463


In [ ]:
#reweight
sample_weight = reweight(
    protected_variable_t, 
    y_train, 
    verbose=False
)
model_reweight = copy(model_xgb)
model_reweight.fit(X_train, y_train, sample_weight=sample_weight)
explainer_reweight = dx.Explainer(
    model_reweight, 
    X_test, 
    y_test, 
    label='XGBClassifier with Reweight mitigation',
    verbose=False
)


fobject_reweight = explainer_reweight.model_fairness(
    protected=protected_variable,
    privileged=privileged_group
)
fobject_reweight.fairness_check()


Bias detected in 2 metrics: PPV, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'male'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
             TPR     ACC       PPV       FPR       STP
female  1.207171  1.1221  0.787004  0.931818  0.576923


In [ ]:
explainer_reweight.model_performance()

,recall,precision,f1,accuracy,auc
XGBClassifier with Reweight mitigation,0.518423,0.79085,0.626294,0.852185,0.908001


In [ ]:
# roc_pivot
protected_variable = X_test.gender_Male.apply(lambda x: "male" if x else "female")
explainer_roc_pivot = roc_pivot(
    copy(explainer), 
    protected_variable, 
    'male',
    verbose=False
)
explainer_roc_pivot.label = 'XGBClassifier with ROC pivot mitigation'

fobject_roc_pivot = explainer_roc_pivot.model_fairness(
    protected=protected_variable,
    privileged=privileged_group
)
fobject_roc_pivot.fairness_check()

Bias detected in 2 metrics: FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'male'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
             TPR      ACC       PPV       FPR       STP
female  0.986434  1.13017  0.954327  0.377778  0.390374


In [ ]:
explainer_roc_pivot.model_performance()

,recall,precision,f1,accuracy,auc
XGBClassifier with ROC pivot mitigation,0.506981,0.800827,0.620892,0.854745,0.907509


In [ ]:
fobject.plot(objects=[fobject_resample, fobject_reweight,fobject_roc_pivot])

